In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
# Import files to use in preprocessing and machine learning
from implementations import *
from proj1_helpers import *
from preprocess import *
from cross_validation import *
from helpers import *
from costs import *

## Load the training data into feature matrix, class labels, and event ids:

In [4]:
# Download train data and supply path here 
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
# Check the array shape of y, tX, and ids
print(y.shape)
print(tX.shape)
print(ids.shape)

(250000,)
(250000, 30)
(250000,)


# Initial Data Analysis

In observing the original training data, we found out that there exists missing data all over tX. The missing data are represented as value -999. Considering that these columns are critical in model training, we cannot simply delete these rows with -999 values. Therefore, we need to process the original training set before model training.

Firstly, we check the columns of tX to obtain an overview of missing data:

In [ ]:
# Check whether the missing values are associated with the classification result
for col in range(tX.shape[1]):
    tX_T = np.transpose(tX)
    
    null = (tX_T[col] == -999)
    null_s = np.logical_and(y >= 0, null)
    null_b = np.logical_and(y < 0, null)
    
    tX_null = tX[null]
    tX_null_s = tX[null_s]
    tX_null_b = tX[null_b]
    
    if (tX_null.shape[0] > 0):
        # Print the percentage of column 'col' having a -999 (missing) value
        print('Column', col, 'has {}% percentage of missing values'.format(tX_null.shape[0] * 100 / tX.shape[0]))

        # Print the conditional probability of P(y = 1|x having -999)
        print('P(y = 1|x having -999) = {:.3f}%'.format(tX_null_s.shape[0] * 100 / tX_null.shape[0]))
        
        # Print the conditional probability of P(y = -1|x having -999)
        print('P(y = -1|x having -999) = {:.3f}% \n'.format(tX_null_b.shape[0] * 100 / tX_null.shape[0]))

We can see that 11 columns contains at least one -999 (missing value). Now we check whether some of the missing values are dependent on the column named 'PRI_jet_num' (column No. 23), since 'PRI_jet_num' has a discrete value range {0, 1, 2, 3} and our observation on the beginning data rows showed a dependency of some missing values to the value of 'PRI_jet_num' column.

In [ ]:
PRI_jet_range = [i for i in range(0, 4)]
PRI_jet_sum = []
PRI_jet_null = []

for value in PRI_jet_range:
    tX_PRI = tX[tX[:, 22] == value]
    
    # Append values of row numbers for different PRI_jet_num, finally sum up to see whether it equals to the length of tX
    PRI_jet_sum.append(len(tX_PRI))
    
    # Count the number of missing columns corresponding to different PRI_jet_num values
    PRI_jet_keys = []
    for i in range (len(tX_PRI)):
        tX_null_cols = np.count_nonzero(tX_PRI[i] == -999, axis = 0)
        PRI_jet_keys.append(tX_null_cols)
    
    PRI_jet_null.append(list(set(PRI_jet_keys)))

    
print("Sum of rows for different PRI_jet_num: {} \n".format(sum(PRI_jet_sum)))

for i in range(4):
    print("PRI_jet_num =", PRI_jet_range[i], "No. of columns having -999 (a missing value):{}".format(PRI_jet_null[i]))

The above analysis showed that one column with -999 (missing value) is independent of the column 'PRI_jet_num', we check the original training set and we can easily find out that the first tX column 'DER_mass_MMC' is independent of 'PRI_jet_num'. 

# Data Preprocessing

Based on the data analysis above, we conduct the following method to pre-process the training data. 

In [7]:
TX, Y, r_ids = data_preprocess(tX, y, ids, replacing='lr', mode='std_norm')

print(TX[0].shape)
print(TX[1].shape)
print(TX[2].shape)

(99913, 18)
(77544, 22)
(72543, 29)


In [5]:
# (Not used)
# Split the database based on 'PRI_jet_num' (0, 1, 2&3)
tX_jet_0, tX_jet_1, tX_jet_2, y_jet_0, y_jet_1, y_jet_2, r_ids = split_reformat_data(tX, y, ids)

#print(tX_jet_0.shape)
#print(tX_jet_1.shape)
#print(tX_jet_2.shape)

In [ ]:
# (Not used)
# Replacing the missing values of first feature column
# 'median', 'mean', or 'lr' (linear regression)
replace_missing_value(tX_jet_0, 0, 'lr')
replace_missing_value(tX_jet_1, 0, 'lr')
replace_missing_value(tX_jet_2, 0, 'lr')

In [ ]:
# (Not used)
# (Use the above block or this block)
# Replacing the missing values by k-means clustering
k_means_replacing(tX_jet_0)
k_means_replacing(tX_jet_1)
k_means_replacing(tX_jet_23)

print((tX_jet_0[tX_jet_0[:, 0]== -999][:,0]).shape)
print((tX_jet_1[tX_jet_1[:, 0]== -999][:,0]).shape)
print((tX_jet_23[tX_jet_23[:, 0]== -999][:,0]).shape)

In [7]:
# (Not used)
# Standardize and/or normalize the splitted training data
tx_0 = std_norm_preprocess(tx_jet_0, 'std_norm')
tx_1 = std_norm_preprocess(tx_jet_1, 'std_norm')
tx_2 = std_norm_preprocess(tx_jet_2, 'std_norm')

TX = [tx_0, tx_1, tx_2]
Y = [y_jet_0, y_jet_1, y_jet_2]

# Model Training

## Least Squares

### Linear Regression with Gradient Descent

In [8]:
# Set H-parameters
GAMMA_LRGD = 0.0005
MAX_ITERS_LRGD = 50

In [11]:
INITIAL_W = np.zeros((TX[0].shape[1]+1, 1))
w_lrgd, loss_lrgd = least_squares_GD(Y[0], TX[0], INITIAL_W, MAX_ITERS_LRGD, GAMMA_LRGD)
    
print('Loss of tx_{},'.format(i), loss_lrgd, 'iterations: {}'.format(MAX_ITERS_LRGD))


MemoryError: Unable to allocate 74.4 GiB for an array with shape (99913, 99913) and data type float64

### Linear Regression with Stochastic Gradient Descent

In [126]:
# Set H-parameters
GAMMA_LRSGD = 0.0005
MAX_ITERS_LRSGD = 200
BATCH_SIZE = 64

In [ ]:
#WEIGHT_LRSGD = []
#LOSS_LRSGD = []
print('Parameters for training:\n')
print('Gamma = {}'.format(GAMMA_LRSGD))
print('Iterations = {}'.format(MAX_ITERS_LRSGD))
print('Batch size = {}'.format(BATCH_SIZE))

# Fine-tuning the data for each splitted training set
INITIAL_W = np.zeros((TX[0].shape[1]+1, 1))
w_lrsgd, loss_lrsgd = least_squares_SGD(Y[0], TX[0], INITIAL_W, MAX_ITERS_LRSGD, 
                                          GAMMA_LRSGD, BATCH_SIZE)

In [ ]:
print('Parameters for training:\n')
print('Gamma = {}'.format(GAMMA_LRSGD))
print('Iterations = {}'.format(MAX_ITERS_LRSGD))
print('Batch size = {}'.format(BATCH_SIZE))

# Fine-tuning the data for each splitted training set
INITIAL_W = np.zeros((TX[1].shape[1]+1, 1))
w_lrsgd_1, loss_lrsgd_1 = least_squares_SGD(Y[1], TX[1], INITIAL_W, MAX_ITERS_LRSGD, 
                                          GAMMA_LRSGD, BATCH_SIZE)

In [ ]:
print('Parameters for training:\n')
print('Gamma = {}'.format(GAMMA_LRSGD))
print('Iterations = {}'.format(MAX_ITERS_LRSGD))
print('Batch size = {}'.format(BATCH_SIZE))

# Fine-tuning the data for each splitted training set
INITIAL_W = np.zeros((TX[2].shape[1]+1, 1))
w_lrsgd_2, loss_lrsgd_2 = least_squares_SGD(Y[2], TX[2], INITIAL_W, MAX_ITERS_LRSGD, 
                                          GAMMA_LRSGD, BATCH_SIZE)

### Least Squares with Normal Equation

In [9]:
# Set H-parameters
DEGREE = np.arange(2, 11)

In [10]:
# Fine-tuning each splitted training dataset
W_LSNE_0 = []
LOSS_LSNE_0 = []

for i in DEGREE:
    poly_tx = build_poly(TX[0], i)
    w_lsne, loss_lsne = least_squares(Y[0], poly_tx)
    print('Degree={a}, Loss={b}'.format(a=i, b=loss_lsne))
    
    W_LSNE_0.append(w_lsne)
    LOSS_LSNE_0.append(loss_lsne)

# Obtain the optimal training weight
w_lsne_opt_0 = W_LSNE_0[np.argmin(LOSS_LSNE_0)]

print('\nOptimal Degree={a}, Loss={b}'.format(a=np.min(DEGREE)+np.argmin(LOSS_LSNE_0),
                                            b=LOSS_LSNE_0[np.argmin(LOSS_LSNE_0)]))

Degree=2, Loss=2.716973059046098
Degree=3, Loss=0.5977268952756707
Degree=4, Loss=0.2446389906141943
Degree=5, Loss=0.2412249117818718
Degree=6, Loss=0.23841392000305464
Degree=7, Loss=0.2687330711970408
Degree=8, Loss=0.277579515434394
Degree=9, Loss=0.23403774649861855
Degree=10, Loss=0.29784256510695334

Optimal Degree=9, Loss=0.23403774649861855


In [11]:
# Fine-tuning each splitted training dataset
W_LSNE_1 = []
LOSS_LSNE_1 = []

for i in DEGREE:
    poly_tx = build_poly(TX[1], i)
    w_lsne, loss_lsne = least_squares(Y[1], poly_tx)
    print('Degree={a}, Loss={b}'.format(a=i, b=loss_lsne))
    
    W_LSNE_1.append(w_lsne)
    LOSS_LSNE_1.append(loss_lsne)

# Obtain the optimal training weight
w_lsne_opt_1 = W_LSNE_1[np.argmin(LOSS_LSNE_1)]

print('\nOptimal Degree={a}, Loss={b}'.format(a=np.min(DEGREE)+np.argmin(LOSS_LSNE_1),
                                            b=LOSS_LSNE_1[np.argmin(LOSS_LSNE_1)]))

Degree=2, Loss=0.3726320941707011
Degree=3, Loss=0.3388298682447978
Degree=4, Loss=0.32928504062212327
Degree=5, Loss=0.32193768701715275
Degree=6, Loss=0.3175146488850505
Degree=7, Loss=0.31260805456110285
Degree=8, Loss=0.3061163557630781
Degree=9, Loss=0.2984343363651026
Degree=10, Loss=0.34563275797245163

Optimal Degree=9, Loss=0.2984343363651026


In [12]:
# Fine-tuning each splitted training dataset
W_LSNE_2 = []
LOSS_LSNE_2 = []

for i in DEGREE:
    poly_tx = build_poly(TX[2], i)
    w_lsne, loss_lsne = least_squares(Y[2], poly_tx)
    print('Degree={a}, Loss={b}'.format(a=i, b=loss_lsne))
    
    W_LSNE_2.append(w_lsne)
    LOSS_LSNE_2.append(loss_lsne)

# Obtain the optimal training weight
w_lsne_opt_2 = W_LSNE_2[np.argmin(LOSS_LSNE_2)]

print('\nOptimal Degree={a}, Loss={b}'.format(a=np.min(DEGREE)+np.argmin(LOSS_LSNE_2),
                                            b=LOSS_LSNE_2[np.argmin(LOSS_LSNE_2)]))

Degree=2, Loss=0.3628695022914208
Degree=3, Loss=0.32984694163527467
Degree=4, Loss=0.3102700012975416
Degree=5, Loss=0.3015283412016949
Degree=6, Loss=0.2985635757978061
Degree=7, Loss=0.29361965896198283
Degree=8, Loss=0.28149784722826043
Degree=9, Loss=0.2694621706236164
Degree=10, Loss=0.2641948086844093

Optimal Degree=10, Loss=0.2641948086844093


In [ ]:
# Test the accuracy on the training set
W_LSNE = [w_lsne_opt_0, w_lsne_opt_1, w_lsne_opt_2]
pred_tr_lsne = data_pred(TX, W_LSNE, step='tr')
accu_lsne = accuracy_pred(pred_tr_lsne, Y, _ids)

In [ ]:
accu_lsne

## Ridge Regression

In [ ]:
# Set H-parameters
K_FOLD = 10
DEGREE = np.arange(1, 7)
SEED = 5
LAMBDA = np.logspace(-6, -3, 30)

In [ ]:
# Find an optimal set of polynomial expansion degree and learning rate (lambda)
# Then calculate the corresponding weight and loss (rmse)
# (based on the H-parameters set above, for each splitted training set)
WEIGHT_RIDGE = []
LOSS_RIDGE = []

for i in range(len(TX)):
    best_deg, best_lambda = find_optimal(Y[i], TX[i], DEGREE, K_FOLD, LAMBDA, SEED)
    print('The best degree of tx_{}:'.format(i), best_deg, 'with lambda:', best_lambda)
    
    poly_tx = build_poly(TX[i], best_deg)
    w, loss = ridge_regression(Y[i], poly_tx, best_lambda)
    
    WEIGHT.append(w)
    LOSS_RIDGE.append(loss)

## Logistic Regression

### Logistic Regression (SGD)

In [2]:
# Set H-parameters
MAX_ITERS_LOGIC = 500
THRESHOLD_LOGIC = 1e-8
GAMMA_LOGIC = 0.005
BATCH_SIZE = 128

In [ ]:
# Reshape label y to 2D-array
Y_LOGIC = []

for i in range(len(Y)):
    Y_LOGIC.append(np.array(Y[i]).reshape(-1, 1)) 

In [ ]:
#WEIGHT_LOGIC = []
#LOSS_LOGIC = []

INITIAL_W = np.zeros((TX[0].shape[1]+1, 1))
w_rlogic_0, loss_rlogic_0 = logistic_regression(Y_LOGIC[0], TX[0], INITIAL_W, MAX_ITERS_LOGIC,
                                                GAMMA_LOGIC, THRESHOLD_LOGIC, BATCH_SIZE)

In [ ]:
INITIAL_W = np.zeros((TX[1].shape[1]+1, 1))
w_rlogic_1, loss_rlogic_1 = logistic_regression(Y_LOGIC[1], TX[1], INITIAL_W, MAX_ITERS_LOGIC,
                                                GAMMA_LOGIC, THRESHOLD_LOGIC, BATCH_SIZE)

In [ ]:
INITIAL_W = np.zeros((TX[2].shape[1]+1, 1))
w_rlogic_2, loss_rlogic_2 = logistic_regression(Y_LOGIC[2], TX[2], INITIAL_W, MAX_ITERS_LOGIC,
                                                GAMMA_LOGIC, THRESHOLD_LOGIC, BATCH_SIZE)

### Regularized Logistic Regression (GD)

In [ ]:
# Set H-parameters
MAX_ITERS_RLOGIC = 1000
THRESHOLD_RLOGIC = 1e-8
GAMMA_RLOGIC = 0.01
BATCH_SIZE = 128

In [ ]:
# Reshape label y to 2D-array
Y_REG_LOGIC = []

for i in range(len(Y)):
    Y_REG_LOGIC.append(np.array(Y[i]).reshape(-1, 1)) 

In [ ]:
# Fine-tuning each splitted dataset
INITIAL_W = np.zeros((TX[0].shape[1], 1))
w_rlogic, loss_rlogic = reg_logistic_regression(Y_REG_LOGIC[0], TX[0], INITIAL_W, MAX_ITERS_RLOGIC,
                                                GAMMA_RLOGIC, THRESHOLD_RLOGIC, BATCH_SIZE)
    
WEIGHT_RLOGIC.append(w)
LOSS_RLOGIC.append(loss)

In [ ]:
INITIAL_W = np.zeros((TX[1].shape[1], 1))
w_rlogic, loss_rlogic = reg_logistic_regression(Y_REG_LOGIC[1], TX[1], INITIAL_W, MAX_ITERS_RLOGIC,
                                                GAMMA_RLOGIC, THRESHOLD_RLOGIC, BATCH_SIZE)
    
WEIGHT_RLOGIC.append(w)
LOSS_RLOGIC.append(loss)

In [ ]:
INITIAL_W = np.zeros((TX[2].shape[1], 1))
w_rlogic, loss_rlogic = reg_logistic_regression(Y_REG_LOGIC[2], TX[2], INITIAL_W, MAX_ITERS_RLOGIC,
                                                GAMMA_RLOGIC, THRESHOLD_RLOGIC, BATCH_SIZE)
    
WEIGHT_RLOGIC.append(w)
LOSS_RLOGIC.append(loss)

## Generate predictions and save ouput in csv format for submission:

In [3]:
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
Y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [10]:
TX_test, _, r_ids_test = data_preprocess(tX_test, Y_test, ids_test, replacing='lr',
                                     mode='std_norm')

print(TX_test[0].shape)
print(TX_test[1].shape)
print(TX_test[2].shape)

(227458, 18)
(175338, 22)
(165442, 29)


In [31]:
# W_tr = []
OUTPUT_PATH = 'data/pred.csv' # TODO: fill in desired name of output file for submission

y_pred = data_pred(TX, W_tr, step='te')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)